In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, utils, layers, models, losses, optimizers, metrics
from tensorflow import keras
from PIL import Image
#from scipy.misc import toimage 版本改為上面那個

seed = 13
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:

def data_scale_down(x_train, x_test, scale):
    num_train = x_train.shape[0] // scale
    num_test = x_test.shape[0] // scale
    x_train = x_train[:num_train].astype(np.float32) // 255.
    x_test = x_test[:num_train]
    return x_train, x_test

def mnist_dataset():
    (x_train,_),(x_test,_) = datasets.mnist.load_data()
    x_train, x_test = data_scale_down(x_train, x_test, scale=10)
    print("x_train.shape",x_train.shape)
    print("x_test.shape",x_test.shape)
    return x_train, x_test

x_train, x_test = mnist_dataset()

x_train.shape (6000, 28, 28)
x_test.shape (6000, 28, 28)


In [11]:
filter_size = 64

def generator():
    model = models.Sequential()
    model.add(layers.Dense(units=3*3*filter_size, activation="relu", name="hd_1"))
    model.add(layers.Reshape(target_shape=[3,3,filter_size], name="reshp"))
    model.add(layers.Conv2DTranspose(filters=filter_size//2, kernel_size=3, strides=2,
                                     activation="relu", padding="valid", name="conv2"))
    model.add(layers.BatchNormalization(name="bn2"))
    model.add(layers.Conv2DTranspose(filters=filter_size//4, kernel_size=4, strides=2,
                                     activation="relu", padding="same", name="conv3"))
    model.add(layers.BatchNormalization(name="bn3"))
    model.add(layers.Conv2DTranspose(filters=1, kernel_size=4, strides=2,
                                     activation="tanh", padding="same", name="conv4"))
    
    return model

class Generator(models.Model):

    def __init__(self):
        super(Generator, self).__init__()
        self.model = generator()        

    def call(self, inputs):        
        return self.model(inputs)


In [14]:

leraning_rate = 0.01
is_training = True
batch_size = 128
z_dim = 100

g_model = Generator()
g_model.build(input_shape=(batch_size,z_dim))
g_model.model.summary()
for layer in g_model.model.layers:
    print(layer.name,"\t", layer)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hd_1 (Dense)                 multiple                  58176     
_________________________________________________________________
reshp (Reshape)              multiple                  0         
_________________________________________________________________
conv2 (Conv2DTranspose)      multiple                  18464     
_________________________________________________________________
bn2 (BatchNormalization)     multiple                  128       
_________________________________________________________________
conv3 (Conv2DTranspose)      multiple                  8208      
_________________________________________________________________
bn3 (BatchNormalization)     multiple                  64        
_________________________________________________________________
conv4 (Conv2DTranspose)      multiple                 

In [19]:
def discriminator():
    # input image is [-1, 28, 28, 1]
    filters = 64
    ker_size = 4
    model = models.Sequential()
    model.add(layers.Conv2D(filters=filters, kernel_size=ker_size, strides=2,
                            activation="relu", padding="same", name="conv1"))
    model.add(layers.Conv2D(filters=filters*2, kernel_size=ker_size, strides=2,
                            activation="relu", padding="same", name="conv2"))
    model.add(layers.BatchNormalization(name="bn2"))
    model.add(layers.Conv2D(filters=filters*4, kernel_size=ker_size, strides=2,
                            activation="relu", padding="same", name="conv3"))
    model.add(layers.BatchNormalization(name="bn3"))
    model.add(layers.Flatten(name="flat"))
    model.add(layers.Dense(units=1, name="logits"))
    
    return model

class Discriminator(models.Model):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = discriminator()        

    def call(self, inputs):        
        return self.model(inputs)

In [20]:
d_model = Discriminator()
d_model.build(input_shape=(batch_size, 28, 28, 1))
d_model.model.summary()
for layer in d_model.model.layers:
    print(layer.name,"\t", layer)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               multiple                  1088      
_________________________________________________________________
conv2 (Conv2D)               multiple                  131200    
_________________________________________________________________
bn2 (BatchNormalization)     multiple                  512       
_________________________________________________________________
conv3 (Conv2D)               multiple                  524544    
_________________________________________________________________
bn3 (BatchNormalization)     multiple                  1024      
_________________________________________________________________
flat (Flatten)               multiple                  0         
_________________________________________________________________
logits (Dense)               multiple                 

In [21]:
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(batch_size*4).batch(batch_size).repeat()
db_iter = iter(dataset) 

In [27]:

def celoss_ones(logits, smooth=0.0):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits, labels=tf.ones_like(logits)*(1.0 - smooth)
    ))

def celoss_zeros(logits, smooth=0.0):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits, labels=tf.zeros_like(logits)*(1.0 - smooth)
    ))

def d_loss_fn(model_g, model_d, input_noise, real_image):
    fake_image = model_g(input_noise) # output shape (128,28,28,1)
    d_real_logits = model_d(real_image) # output shape (128,1)
    d_fake_logits = model_d(fake_image) # output shape (128,1)
    
    d_loss_real = celoss_ones(d_real_logits, smooth=0.1)
    d_loss_fake = celoss_zeros(d_fake_logits, smooth=0.0)
    loss = d_loss_real + d_loss_fake
    return loss

def g_loss_fn(model_g, model_d, input_noise):
    fake_image = model_g(input_noise) # output shape (128,28,28,1)
    d_fake_logits = model_d(fake_image) # output shape (128,1)
    loss = celoss_ones(d_fake_logits, smooth=0.1)
    return loss

def save_result(val_out, val_block_size, image_fn, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    Image.fromarray(final_image, mode=color_mode).save(image_fn)
    

assets_dir = './images'
if not os.path.isdir(assets_dir):
    os.makedirs(assets_dir)
val_block_size = 10
val_size = val_block_size * val_block_size

In [29]:

epochs = 100
learning_rate = 0.01
inputs_shape = [-1, 28, 28, 1]
is_training = True

g_optimizer = optimizers.Adam(learning_rate=learning_rate)
d_optimizer = optimizers.Adam(learning_rate=learning_rate)

#g_model.compile(optimizer=g_optimizer, loss="categorical_crossentropy")
#d_model.compile(optimizer=d_optimizer, loss="categorical_crossentropy")

def train():
    for epoch in range(epochs):
        batch_x = next(db_iter)
        batch_x = tf.reshape(batch_x, shape=inputs_shape)
        batch_x = batch_x * 2.0 -1.0 # -1 < px_value < 1
        
        # Sample random noise for G
        batch_z = tf.random.uniform(shape=[batch_size, z_dim], minval=-1., maxval=1.)
        
        with tf.GradientTape() as tape:
            d_loss = d_loss_fn(g_model.model, d_model.model, batch_z, batch_x)
        grads = tape.gradient(d_loss, d_model.model.trainable_variables)
        d_optimizer.apply_gradients(zip(grads, d_model.model.trainable_variables))
        
        with tf.GradientTape() as tape:
            g_loss = g_loss_fn(g_model.model, d_model.model, batch_z)
        grads = tape.gradient(g_loss, g_model.model.trainable_variables)
        g_optimizer.apply_gradients(zip(grads, g_model.model.trainable_variables))
        
        if (epoch+1) % 10 == 0:
            print((epoch+1), 'd loss:', float(d_loss), 'g loss:', float(g_loss))
            val_z = np.random.uniform(-1, 1, size=(val_size, z_dim))
            fake_image = g_model.predict(val_z)
            image_fn = os.path.join('images', 'gan-val-{:03d}.png'.format(epoch + 1))
            save_result(fake_image, val_block_size, image_fn, color_mode='L')
    print("finish training")
            
train()

9 d loss: 1.4788918495178223 g loss: 0.6567821502685547
19 d loss: 1.4440267086029053 g loss: 0.7294305562973022
29 d loss: 1.3646976947784424 g loss: 0.9330277442932129
39 d loss: 1.3927652835845947 g loss: 0.6837785840034485
49 d loss: 1.3176616430282593 g loss: 0.843607485294342
59 d loss: 1.4530634880065918 g loss: 1.5772510766983032
69 d loss: 1.533296823501587 g loss: 0.8039431571960449
79 d loss: 1.3543953895568848 g loss: 0.575875461101532
89 d loss: 1.2793517112731934 g loss: 0.7645173668861389
99 d loss: 1.4043993949890137 g loss: 1.1818479299545288
